<a href="https://colab.research.google.com/github/afsarahannan/Speech2Text/blob/main/bengali_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook combines Helsinki NLP to tokenize and translate english language to bengali and then summarize the data. The output is then directly translated using Deep translate from Google instead of a neural network to avoid misconstrued sentences 

In [1]:
!pip install transformers #use this only once to download it in your colab environment 
import transformers #access the modules with the prefix 'transformers.module_name'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#the normalizer library is used for languages that derive from Sanskrit mainly for languages such as Hindi, Bengali, Telegu and more 
# !pip install normalizer 
!pip install indic-nlp-library
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#The Helsinki tokenizer has dependencies on sentencepiece for tokenization of bengali language 
!pip install sentencepiece
import sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# load the pre-trained model for translation
from transformers import MarianMTModel  
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_bn_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bn-en")
model_bn_en = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-bn-en")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
#load the pre-trained model and tokenizer for summarization 
from transformers import T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
#to translate Bengali language back to english directly without any further tuning 
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
#Torch script to test the model 
from deep_translator import GoogleTranslator
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("input:")
input_text = input()

with torch.no_grad():
    # tokenize the input text
    input_ids = tokenizer_bn_en.encode(input_text, return_tensors='pt')

    # generate the output translation
    output_ids = model_bn_en.generate(input_ids)
    output_text1 = tokenizer_bn_en.decode(output_ids[0], skip_special_tokens=True)

    #input for the summarization model 
    input_text2 = output_text1

    # tokenize the input text and generate the summary in english 
    input_ids = tokenizer.encode(input_text2, return_tensors='pt')
    summary_ids = model.generate(input_ids, max_length=50, num_beams=30, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


print("\noutput:\n" + GoogleTranslator(source='en', target='bn').translate(summary))

input:
বেঙ্গলি ভাষার মূল্যায়ন এবং অনুবাদ সমস্যাগুলি দূরত্বপূর্ণ এবং চ্যালেঞ্জিং হতে পারে। এটি একটি ভিন্নধর্মী লিপি ব্যবহার করে যা কম্পিউটার জন্য অস্বস্তি তৈরি করতে পারে। এছাড়াও, বেঙ্গলি ভাষার বক্তৃতা একটি সাধারণ উচ্চারণ নিয়ে না বরং বিভিন্ন উচ্চারণ এবং শব্দ ব্যবহার করে একটি বিশাল ভূমিকা রাখে। এছাড়াও, বেঙ্গলি ভাষায় বক্তৃতা আমার মতে একটি সমস্যা হল কথাগুলির মানের একটি নির্দিষ্টকরণ না থাকা। কম্পিউটার বেঙ্গলি ভাষা বুঝতে অস্বস্ত হওয়ার কারণে সেটি সঠিকভাবে অনুবাদ করতে সমস্যাগুলি উঠে আসে। সম্পূর্ণ স্বয়ংক্রিয় ভাষান্তর সফটওয়্যার প্রয়োগ করলেও এই সমস্যাগুলি নিজের দেশের ভাষার মতন সম্পূর্ণ উপস্থাপন না করে সেটির সাথে সামঞ্জস


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



output:
বাংলা ভাষা মূল্যায়ন এবং অনুবাদ সমস্যা একটি দীর্ঘ এবং চ্যালেঞ্জিং সমস্যা হতে পারে। তদুপরি, বাংলায় ধ্বনি বক্তৃতার কোনো সাধারণ ভাষা নেই বরং শব্দ ও শব্দের বিরাট ভূমিকা রয়েছে।
